# Résolution de systèmes linéaires et inversion

Nous considérons le système linéaire
$$
Ax = b
$$
où $A \in \mathbb{R}^{n \times n}$, $b \in mathbb{R}^n$, et nous cherchons à déterminer le vecteur $x \in \mathbb{R}^n$. Nous supposons de plus que le rang de $A$ vaut $n$.

In [ ]:
using LinearAlgebra
using BenchmarkTools

## Temps de calul

Nous allons créer un matrice test qui nous servira à calculer les temps de calcul en utilisant l'inversion matricielle ou les techniques de factorisation. L'inversion prend $O(n^3)$ opérations, tandis que la factorisation requiert $O(n^2)$ opérations, où $n$ est la dimension de $A$, $b$ et $x$.

In [ ]:
n = 10000

In [ ]:
A = zeros(n,n)
for i = 1:n
    A[i,i] = 2.0
end
for i = 1:n-1
    A[i,i+1] = A[i+1,i] = -1.0
end

In [ ]:
A

In [ ]:
inv(A)

Nous pouvons déjà remarque que $A$ est creuse, alors que son inverse est dense.

Créons le membre de droite du système.

In [ ]:
b = ones(n)

Nous résolvons à présent le système avec les deux techniques.

In [ ]:
x1 = inv(A)*b

In [ ]:
x2 = A\b

Comparons la précision des résultats.

In [ ]:
[norm(A*x1-b), norm(A*x2-b)]

Nous voyons que la technique de factorisation est légèremement plus précise.

Comparons les temps d'exécution.

In [ ]:
@benchmark inv(A)*b

In [ ]:
@benchmark A\b

Nous allons à présent exploiter le caractère creux de $A$.

In [ ]:
using SparseArrays

In [ ]:
A2 = sparse(A)

In [ ]:
inv(A2)

Nous voyons que Julia détecte que l'opération d'inversion serait inefficace au niveau mémoire.

In [ ]:
x3 = A2\b

In [ ]:
@benchmark A2\b

In [ ]:
norm(A*x3-b)

La précision reste similaire au cas dense, mais le temps d'exécution est significativement plus faible.

## Précision des résultats

Nous allons modifier la diagonale de la matrice pour accentuer les résultats.

In [ ]:
for i = 1:n
    A[i,i] = 10.0^(-i)
end

A

Nous calculons le système suivant les deux techniques.

In [ ]:
x1 = inv(A)*b

In [ ]:
x2 = A\b

Calculons aussi en utilisant une matrice creuse.

In [ ]:
A2 = sparse(A)

In [ ]:
x3 = A2\b

In [ ]:
[norm(A*x1-b) norm(A*x2-b) norm(A*x3-b)]

À nouveau, nous voyons que la technique d'inversion donne des résultats moins intéressants.